In [1]:
# url = "https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv"
# !wget $url

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Data Prepration 

In [3]:
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.columns[df.dtypes=='object'])

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

In [5]:
df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')

In [6]:
df.totalcharges = df.totalcharges.fillna(0)

In [7]:
df.churn = (df.churn == 'yes').astype(int)

# Setting up the validation framework

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [10]:
len(df_full_train), len(df_test)

(5634, 1409)

In [11]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [12]:
len(df_train), len(df_val), len(df_test)

(4225, 1409, 1409)

In [13]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [14]:
y_train = df_train.churn.values
y_val = df_test.churn.values
y_test = df_val.churn.values

In [15]:
del df_train['churn']
del df_val['churn']
del df_test['churn']

# EDA

In [16]:
df_full_train = df_full_train.reset_index(drop=True)

In [17]:
df_full_train.churn.value_counts(normalize=True)

0    0.730032
1    0.269968
Name: churn, dtype: float64

In [18]:
global_churn_rate = df_full_train.churn.mean()
round(global_churn_rate, 2)

0.27

In [19]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

In [20]:
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
       'phoneservice', 'multiplelines', 'internetservice','onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod']

In [21]:
df_full_train[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

# Feature importcance : Churn rate and risk ratio

In [22]:
df_full_train.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,5442-pptjy,male,0,yes,yes,12,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,19.70,258.35,0
1,6261-rcvns,female,0,no,no,42,yes,no,dsl,yes,...,yes,yes,no,yes,one_year,no,credit_card_(automatic),73.90,3160.55,1
2,2176-osjuv,male,0,yes,no,71,yes,yes,dsl,yes,...,no,yes,no,no,two_year,no,bank_transfer_(automatic),65.15,4681.75,0
3,6161-erdgd,male,0,yes,yes,71,yes,yes,dsl,yes,...,yes,yes,yes,yes,one_year,no,electronic_check,85.45,6300.85,0
4,2364-ufrom,male,0,no,no,30,yes,no,dsl,yes,...,no,yes,yes,no,one_year,no,electronic_check,70.40,2044.75,0


In [23]:
churn_female = df_full_train[df_full_train.gender == 'female'].churn.mean()
churn_female

0.27682403433476394

In [24]:
churn_male = df_full_train[df_full_train.gender == 'male'].churn.mean()
churn_male

0.2632135306553911

In [25]:
global_churn = df_full_train.churn.mean()
global_churn

0.26996805111821087

In [26]:
df_full_train.partner.value_counts()

no     2932
yes    2702
Name: partner, dtype: int64

In [27]:
churn_partner = df_full_train[df_full_train.partner == 'yes'].churn.mean()
churn_partner

0.20503330866025166

In [28]:
global_churn - churn_partner

0.06493474245795922

In [29]:
churn_no_partner = df_full_train[df_full_train.partner == 'no'].churn.mean()
churn_no_partner

0.3298090040927694

### Risk ratio

In [30]:
churn_no_partner / global_churn
# here people with no partner are 22% more likely to churn

1.2216593879412643

In [31]:
churn_partner / global_churn
# people with partner are 25% less likely to churn

0.7594724924338315

In [32]:
from IPython.display import display

In [33]:
for c in categorical:
    print(c)
    df_group = df_full_train.groupby(c).churn.agg(['mean', 'count'])
    df_group['diff'] = df_group['mean'] - global_churn
    df_group['risk'] = df_group['mean'] / global_churn
    display(df_group)
    print()
    print()

gender


,mean,count,diff,risk
gender,,,,
female,0.276824,2796,0.006856,1.025396
male,0.263214,2838,-0.006755,0.974980




seniorcitizen


,mean,count,diff,risk
seniorcitizen,,,,
0,0.242270,4722,-0.027698,0.897403
1,0.413377,912,0.143409,1.531208




partner


...

,mean,count,diff,risk
contract,,,,
month-to-month,0.431701,3104,0.161733,1.599082
one_year,0.120573,1186,-0.149395,0.446621
two_year,0.028274,1344,-0.241694,0.104730




paperlessbilling


,mean,count,diff,risk
paperlessbilling,,,,
no,0.172071,2313,-0.097897,0.637375
yes,0.338151,3321,0.068183,1.252560




paymentmethod


,mean,count,diff,risk
paymentmethod,,,,
bank_transfer_(automatic),0.168171,1219,-0.101797,0.622928
credit_card_(automatic),0.164339,1217,-0.105630,0.608733
electronic_check,0.455890,1893,0.185922,1.688682
mailed_check,0.193870,1305,-0.076098,0.718121


# Feature Importance: Mutual Information
* Mutual Information quantifies the amount of information obtained abount one random variable by observing the other random variable

In [34]:
from sklearn.metrics import mutual_info_score

In [35]:
mutual_info_score(df_full_train.churn, df_full_train.contract)

0.0983203874041556

In [36]:
mutual_info_score(df_full_train.churn, df_full_train.gender)

0.00011748462111432767

In [37]:
def mutual_info_churn_score(series):
    return mutual_info_score(series, df_full_train.churn) * 1000

In [38]:
mi = df_full_train[categorical].apply(mutual_info_churn_score)
mi.sort_values(ascending=False)

contract            98.320387
onlinesecurity      63.085250
techsupport         61.032460
internetservice     55.867946
onlinebackup        46.923464
deviceprotection    43.452869
paymentmethod       43.210028
streamingtv         31.853331
streamingmovies     31.580897
paperlessbilling    17.588827
dependents          12.345815
partner              9.967689
seniorcitizen        9.410216
multiplelines        0.857448
phoneservice         0.228713
gender               0.117485
dtype: float64

# Feature Importance: Correlation

In [39]:
df_full_train[numerical].corrwith(df_full_train.churn)

tenure           -0.351885
monthlycharges    0.196805
totalcharges     -0.196353
dtype: float64

In [40]:
df_full_train[(df_full_train.tenure > 2) & (df_full_train.tenure <=12)].churn.mean()

0.3994413407821229

# One-hot encoding

In [41]:
from sklearn.feature_extraction import DictVectorizer

In [42]:
dv = DictVectorizer(sparse=False)

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

# Training Logistic Regression 

In [43]:
from sklearn.linear_model import LogisticRegression

In [44]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [45]:
model.intercept_[0]

-0.10903522099393054

In [46]:
model.coef_[0].round(3)

array([ 0.475, -0.175, -0.408, -0.03 , -0.078,  0.063, -0.089, -0.081,
       -0.034, -0.073, -0.335,  0.316, -0.089,  0.004, -0.258,  0.141,
        0.009,  0.063, -0.089, -0.081,  0.266, -0.089, -0.284, -0.231,
        0.124, -0.166,  0.058, -0.087, -0.032,  0.07 , -0.059,  0.141,
       -0.249,  0.215, -0.12 , -0.089,  0.102, -0.071, -0.089,  0.052,
        0.213, -0.089, -0.232, -0.07 ,  0.   ])

In [47]:
y_pred = model.predict_proba(X_val)[:, 1]

In [48]:
churn_decision = (y_pred >= 0.5)

In [49]:
(y_val == churn_decision).mean()

0.6458481192334989

In [50]:
df_pred = pd.DataFrame()
df_pred['probability'] = y_pred
df_pred['predictions'] = churn_decision.astype(int)
df_pred['actual'] = y_val

In [51]:
df_pred['correct'] = (df_pred.predictions == df_pred.actual)

In [52]:
df_pred.correct.mean()

0.6458481192334989

# Model Interpretation

In [73]:
dict(zip(dv.get_feature_names_out(), model.coef_[0].round(3)))

{'contract=month-to-month': 0.268,
 'contract=one_year': -0.153,
 'contract=two_year': -0.238,
 'dependents=no': 0.029,
 'dependents=yes': -0.153,
 'deviceprotection=no': 0.066,
 'deviceprotection=no_internet_service': -0.092,
 'deviceprotection=yes': -0.098,
 'gender=female': -0.052,
 'gender=male': -0.071,
 'internetservice=dsl': -0.298,
 'internetservice=fiber_optic': 0.266,
 'internetservice=no': -0.092,
 'monthlycharges': 0.004,
 'multiplelines=no': -0.221,
 'multiplelines=no_phone_service': 0.032,
 'multiplelines=yes': 0.066,
 'onlinebackup=no': 0.089,
 'onlinebackup=no_internet_service': -0.092,
 'onlinebackup=yes': -0.121,
 'onlinesecurity=no': 0.232,
 'onlinesecurity=no_internet_service': -0.092,
 'onlinesecurity=yes': -0.263,
 'paperlessbilling=no': -0.269,
 'paperlessbilling=yes': 0.145,
 'partner=no': -0.067,
 'partner=yes': -0.056,
 'paymentmethod=bank_transfer_(automatic)': -0.086,
 'paymentmethod=credit_card_(automatic)': -0.105,
 'paymentmethod=electronic_check': 0.261,

In [54]:
small = ['contract', 'tenure', 'monthlycharges']

In [55]:
df_train[small].iloc[:10].to_dict(orient='records')

[{'contract': 'two_year', 'tenure': 72, 'monthlycharges': 115.5},
 {'contract': 'month-to-month', 'tenure': 10, 'monthlycharges': 95.25},
 {'contract': 'month-to-month', 'tenure': 5, 'monthlycharges': 75.55},
 {'contract': 'month-to-month', 'tenure': 5, 'monthlycharges': 80.85},
 {'contract': 'two_year', 'tenure': 18, 'monthlycharges': 20.1},
 {'contract': 'month-to-month', 'tenure': 4, 'monthlycharges': 30.5},
 {'contract': 'month-to-month', 'tenure': 1, 'monthlycharges': 75.1},
 {'contract': 'month-to-month', 'tenure': 1, 'monthlycharges': 70.3},
 {'contract': 'two_year', 'tenure': 72, 'monthlycharges': 19.75},
 {'contract': 'month-to-month', 'tenure': 6, 'monthlycharges': 109.9}]

In [56]:
dicts_train_small = df_train[small].to_dict(orient='records')
dicts_val_small = df_val[small].to_dict(orient='records')

In [57]:
dv_small = DictVectorizer(sparse=False)
dv_small.fit(dicts_val_small)

DictVectorizer(sparse=False)

In [58]:
dv_small.get_feature_names_out()

array(['contract=month-to-month', 'contract=one_year',
       'contract=two_year', 'monthlycharges', 'tenure'], dtype=object)

In [59]:
X_train_small = dv_small.transform(dicts_train_small)

In [60]:
model_small = LogisticRegression()
model_small.fit(X_train_small, y_train)

LogisticRegression()

In [61]:
b = model_small.intercept_
b

array([-2.47677566])

In [62]:
w = model_small.coef_[0]
w.round(3)

array([ 0.97 , -0.025, -0.949,  0.027, -0.036])

In [63]:
dict(zip(dv_small.get_feature_names_out(), w.round(3)))

{'contract=month-to-month': 0.97,
 'contract=one_year': -0.025,
 'contract=two_year': -0.949,
 'monthlycharges': 0.027,
 'tenure': -0.036}

# Using the model

In [64]:
dicts_full_train = df_full_train[categorical + numerical].to_dict(orient='records')

In [65]:
dv = DictVectorizer(sparse=False)
X_full_train  = dv.fit_transform(dicts_full_train)

In [66]:
y_full_train = df_full_train.churn.values

In [67]:
model = LogisticRegression()
model.fit(X_full_train, y_full_train)

LogisticRegression()

In [68]:
dicts_test = df_test[categorical + numerical].to_dict(orient='records')

In [69]:
X_test = dv.transform(dicts_test)

In [70]:
y_pred = model.predict_proba(X_test)[:, 1]

In [71]:
churn_decision = (y_pred >= 0.)

In [72]:
(churn_decision == y_test).mean()

0.2739531582682754

In [82]:
customer = dicts_test[-1]
customer

{'gender': 'female',
 'seniorcitizen': 0,
 'partner': 'yes',
 'dependents': 'yes',
 'phoneservice': 'yes',
 'multiplelines': 'yes',
 'internetservice': 'fiber_optic',
 'onlinesecurity': 'yes',
 'onlinebackup': 'no',
 'deviceprotection': 'yes',
 'techsupport': 'no',
 'streamingtv': 'yes',
 'streamingmovies': 'yes',
 'contract': 'month-to-month',
 'paperlessbilling': 'yes',
 'paymentmethod': 'electronic_check',
 'tenure': 17,
 'monthlycharges': 104.2,
 'totalcharges': 1743.5}

In [83]:
X_small = dv.transform([customer])

In [84]:
model.predict_proba(X_small)[0, 1]

0.5968852088299491

In [86]:
y_test[-1]

1